# ASCII para el MTM-SSA toolkit

Script con el objetivo de crear un archivo ascii como datos de entrada para la aplicación de singlar spectrum analysis mediante el toolkit:
https://dept.atmos.ucla.edu/tcd/ssa-mtm-toolkit

En la sección Documentation > Demonstration se detalla que la serie temporal debe presentarse como tabla en un archivo ascii de la forma:

| time | data |

La serie temporal se obtiene a partir de datos de precipitación acumulada mensual de la base de datos GPCC. Los pasos a seguir son:
1. Lectura de datos y recorte espacial enfocado en el centro de Argentina. 
2. Remuestreo de datos, de acumulados mensuales a estacionales. Se guarda todo por separado en un diccionario.
3. Calculo de anomalías respecto del período 1951-1980
4. Promedio regional con peso según latitud (se van las dimensiones lat y lon, por lo que queda una serie 1D)
5. Se pasa de xarray.DataArray a pandas.Dataframe

Luego se crea una tabla ascii (un csv por estación) usando la funcion to_csv() de pandas. 






## Librerias

In [1]:
import xarray as xr
import numpy as np

## Constantes

In [2]:
# Para serie temporal de anomalias CENTRO ARGENTINA
LAT_SUR_M=-35
LAT_NOR_M=-26
LON_OESTE_M=294
LON_ESTE_M=299

#Para calculos CENTRO ARGENTINA
LAT_SUR=-42
LAT_NOR=-16
LON_OESTE=287
LON_ESTE=306

#Periodo largo
ANIO_MIN1=1930
ANIO_MAX1=2016

#Periodo corto
ANIO_MIN2=1981
ANIO_MAX2=2010

#Periodo en formato
TIMEMIN1 = str(ANIO_MIN1)+'-01-01'
TIMEMAX1 = str(ANIO_MAX1)+'-12-31'

#Periodo en formato
TIMEMIN2 = str(ANIO_MIN2)+'-01-01'
TIMEMAX2 = str(ANIO_MAX2)+'-12-31'

#Periodo en el cual se calculará la media para las anomalías
TIMEMIN_MEDIA = '1951-01-01'
TIMEMAX_MEDIA = '1980-12-31'

#Path a los datos
DATOS_PP_GPCC='/datos/Datos_Dalia/precip.mon.total.v2018.nc'

#Path para guardar figuras
SALIDAS='/home/dalia.panza/Proy_IAI/Salidas/SSA/'

#Vector con nombres de las estaciones del año
KEYS=['Verano (DJF)', 'Otoño (MAM)', 'Invierno (JJA)', 'Primavera (SON)']

## Selección de período

In [3]:
per=1

if (per==1):
    
    TIMEMIN=TIMEMIN1
    TIMEMAX=TIMEMAX1
    PER='(1930-2016)'
    
else:
    
    TIMEMIN=TIMEMIN2
    TIMEMAX=TIMEMAX2
    PER='(1981-2010)'

## Lectura de datos

In [4]:
# Abro el archivo usando xarray
ds_pp = xr.open_dataset(DATOS_PP_GPCC)

#Recorte temporal y espacial
pp_recorte = ds_pp.sel(lat=slice(LAT_NOR, LAT_SUR), lon=slice(LON_OESTE,LON_ESTE))['precip']

del ds_pp

#Remuestreo los datos dividiendo por estaciones (DEF-MAM-JJA-SON)
pp_est = pp_recorte.resample(time="QS-DEC").sum()

pp_est={KEYS[0]:pp_est.sel(time=pp_est["time.month"]==12),
        KEYS[1]:pp_est.sel(time=pp_est["time.month"]==3),
        KEYS[2]:pp_est.sel(time=pp_est["time.month"]==6),
        KEYS[3]:pp_est.sel(time=pp_est["time.month"]==9)}

#Excluyo el primer y ultimo datos de verano (Tienen meses creados que no estan en los datos)
pp_est[KEYS[0]]=pp_est[KEYS[0]].sel(time=slice(pp_est[KEYS[0]]['time'][1],
                                                pp_est[KEYS[0]]['time'][-2]))


## Promedio regional de anomalías (3D --> 1D)

In [5]:
########CALCULO DE ANOMALÍAS############

#Anomalía para cada punto de reticula

"""Para la serie original"""
pp_media = pp_recorte.sel(lat=slice(LAT_NOR_M,LAT_SUR_M),
                               lon=slice(LON_OESTE_M,LON_ESTE_M),
                               time=slice(TIMEMIN_MEDIA,TIMEMAX_MEDIA)).mean()
pp_anom =pp_recorte.sel(time=slice(TIMEMIN,TIMEMAX)) - pp_media

"""Por estación"""
#Inicializo lista vacia 
pp_est_anom=[]

#Ciclo para iterar en el diccionario de datos de pp
for key in KEYS:
    #Hago un recorte espacial para el centro de Arg y el periodo 51-80
    pp_media = pp_est[key].sel(lat=slice(LAT_NOR_M,LAT_SUR_M),
                               lon=slice(LON_OESTE_M,LON_ESTE_M),
                               time=slice(TIMEMIN_MEDIA,TIMEMAX_MEDIA)).mean()
    #A la lista vacía le agrego la anomalía para la estación correspondiente
    pp_est_anom.append(pp_est[key].sel(time=slice(TIMEMIN,TIMEMAX))-pp_media)
    
    del pp_media

#Paso de list a dict 
pp_est_anom= dict(zip(KEYS, pp_est_anom))

"""Anual"""
pp_ann_anom=pp_anom.resample(time="Y").sum()

############PROMEDIO REGIONAL################

#Promedio regional con pesos segun latitud
weights = np.cos(np.deg2rad(pp_recorte.lat))

"""Para la serie original"""
pp_anom_PromReg = pp_anom.sel(lat=slice(LAT_NOR_M,LAT_SUR_M),
                          lon=slice(LON_OESTE_M,LON_ESTE_M)).weighted(weights).mean(("lon", "lat"))
pp_anom_PromReg = pp_anom_PromReg.to_dataframe()
"""Por estación"""
#Inicializo lista vacía
pp_anom_est_PromReg=[]

#Data array con valores entre 0 y 1 para hacer promedio con peso x latitud

for key in KEYS:
    da_weighted = pp_est_anom[key].sel(lat=slice(LAT_NOR_M,LAT_SUR_M),
                                      lon=slice(LON_OESTE_M,LON_ESTE_M)).weighted(weights)
    pp_anom_est_PromReg.append(da_weighted.mean(("lon", "lat")))

                      
#Paso de list a dict
pp_anom_est_PromReg= dict(zip(KEYS, pp_anom_est_PromReg))

#del pp_recorte, ds_weighted, pp_est_anom, pp_est

#Paso los DataArrays a pandas DataFrames
for key in KEYS:
    pp_anom_est_PromReg[key] = pp_anom_est_PromReg[key].to_dataframe()


"""Anual"""
pp_ann_anom_PromReg = pp_ann_anom.sel(lat=slice(LAT_NOR_M,LAT_SUR_M),
                          lon=slice(LON_OESTE_M,LON_ESTE_M)).weighted(weights).mean(("lon", "lat"))

pp_ann_anom_PromReg = pp_ann_anom_PromReg.to_dataframe()

## Creación de ASCII

In [9]:
print(pp_anom_est_PromReg[KEYS[0]]['precip'])

for key in KEYS:
    
    path = SALIDAS + 'ascii_ssa_toolkit_{}.csv'.format(key[-4:-1])
   
    pp_anom_est_PromReg[key]['precip'].to_csv(path, header=False, encoding='ascii', index=False)
    
    

time
1930-12-01    103.963638
1931-12-01     19.145315
1932-12-01    -36.733856
1933-12-01   -117.120476
1934-12-01    -67.323784
                 ...    
2011-12-01    -68.633232
2012-12-01    -82.130913
2013-12-01     41.520748
2014-12-01    126.656120
2015-12-01     88.353111
Name: precip, Length: 86, dtype: float32
